<a href="https://colab.research.google.com/github/vlozg/speech_hmm/blob/main/Test_DiagHMM_012.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LƯU Ý: NOTEBOOK NÀY CHỈ DÙNG ĐỂ SHOW KẾT QUẢ CHẠY, KHÔNG NÊN CHẠY LẠI NOTEBOOK NÀY 
(VÌ TRONG NÀY CÓ CODE LƯU LẠI PRETRAINED MODEL LÊN DRIVE SẼ BỊ XÓA)

# Speech to text with HMM

- **Bài toán**: Chuyển giọng nói thành văn bản
    - **Input**: Đoạn ghi âm chứa nội dung là các số từ 0 đến 9
    - **Output**: Phân lớp của đoạn ghi âm

# Các biến thiết lập cho thử nghiệm

In [ ]:
n_mfcc_ceptrum = 20
n_delta_features = 1
n_mixtures = 1
fsdd_split = 0.4
wolfram_split = 0
experiment_id = '012'

In [ ]:
n_mfcc_features = n_mfcc_ceptrum * (1+n_delta_features)
n_mfcc_features

40

# Import và cài đặt thư viện

In [ ]:
# cài lib. note: cài xong phải restart runtime
!pip install pydub
!pip install pomegranate

In [ ]:
# Xác thực google để upload/download qua google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Quản lý file, folder
import os
from shutil import copyfile, rmtree
import random

# Xử lý audio
import librosa
import librosa.display
from scipy.io import wavfile

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pomegranate # Thư viện cho mô hình xác suất
from pomegranate import *

# Tải dữ liệu và lấy xác thực Google

Dữ liệu dùng để huấn luyện và đánh giá, còn xác thực google thì dùng để upload/download mô hình trên drive.

In [ ]:
%%capture
# download wolfram
if not os.path.isfile('./dataset_1_wolfram.zip'):
  !gdown --id 115tIAitBNeJC0DwrP-ZyJ6RS3TyWN0qD
  !unzip -o dataset_1_wolfram.zip

# dowload FSDD
if not os.path.isfile('./dataset_2_FSDD.zip'):
  !gdown --id 1Ua9zlPBc0Fv4xGHSQTb7eIvUh_dqFI6P
  !unzip -o dataset_2_FSDD.zip

# download self recorded audio
!gdown --id 1lH_k1AYMVlJvodtZdD7OK2zkdPXxlW9i

In [ ]:
# Lấy xác thực google để upload/download file
auth.authenticate_user()
gauth =  GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Hàm xử lý âm thanh

In [ ]:
def scaleAddNoise(wave):
  return ((wave - wave.mean()) / wave.std()) + 0.2*np.random.normal(size=wave.shape)

def read_process_audio(full_audio_path):
  # Bắt buộc âm thanh đọc vào phải cùng sample rate
  sample_rate, wave =  wavfile.read(full_audio_path)
  if sample_rate != 16000:
    wave, sample_rate =  librosa.load(full_audio_path, sr=16000) # Hàm đọc của librosa quá chậm, scipy nhanh gấp 5-10 lần
  else:
    wave=wave/32768 # Chuẩn hóa về số thực
  wave = np.pad(wave, (2000,2000), 'constant', constant_values=(0.0,0.0))
  wave = scaleAddNoise(wave)
  return wave, sample_rate

def unvoiced_frame(wave, sample_rate, min_len = 10):
  rms = librosa.feature.rms(y=wave)[0]
  r_normalized = (rms - np.mean(rms)) / np.std(rms)
  p = np.exp(r_normalized) / (1 + np.exp(r_normalized))
  thresh = 0.4
  slice_ = p > thresh
  while (slice_.sum() < 10):
    thresh-=0.05
    slice_ = p > thresh
  return slice_

def extract_mfcc(wave, sample_rate, trim=True):
  S = librosa.feature.melspectrogram(y=wave, sr=sample_rate, n_mels=40)
  if trim:
    S = S[:,unvoiced_frame(wave, sample_rate)]
  mfccs = librosa.feature.mfcc(S=librosa.power_to_db(S), n_mfcc=n_mfcc_ceptrum+2, lifter=40)[2:,:].T/800
  return mfccs

def mfcc_delta_features(mfcc, order):
  if order==0:
    return mfcc
  dmfcc = librosa.feature.delta(mfcc, order=order)
  return dmfcc

def full_mfcc_from_file(full_audio_path, trim=True):
  wave, sr = read_process_audio(full_audio_path)
  mfccs = extract_mfcc(wave, sr, trim)
  full_mfccs = mfccs
  if n_delta_features >= 1:
    mfccs_d1 = mfcc_delta_features(mfccs, 1)
    full_mfccs = np.hstack([full_mfccs,mfccs_d1])
  if n_delta_features >= 2:
    mfccs_d2 = mfcc_delta_features(mfccs, 2)
    full_mfccs = np.hstack([full_mfccs,mfccs_d2])
  return full_mfccs

# Hàm bổ trợ

In [ ]:
def buildDataSet(dir, trim=True):
    # Filter out the wav audio files under the dir
    fileList = [f for f in os.listdir(dir) if os.path.splitext(f)[1] == '.wav']
    dataset = {}
    for fileName in fileList:
        tmp = fileName.split('.')[0]
        label = tmp.split('_')[1]

        # label = filename.split('_')[0]
        feature = full_mfcc_from_file(dir+fileName, trim)
        if label not in dataset.keys():
            dataset[label] = []
            dataset[label].append(feature)
        else:
            exist_feature = dataset[label]
            exist_feature.append(feature)
            dataset[label] = exist_feature
    return dataset

def makeTrainTestDir(mainFolder, filenames, rate):
    paths = [f'{mainFolder}',
             f'{mainFolder}/train_audio',
             f'{mainFolder}/test_audio']

    for path in paths:
        try:
            os.mkdir(path)
        except:
            rmtree(path)
            os.mkdir(path)

    random.seed(1)
    random.shuffle(filenames)
    splitPoint = int(len(filenames)*rate)
    trainFilenames = filenames[:splitPoint]
    testFilenames = filenames[splitPoint:]

    for filename in trainFilenames:
        copyfile(filename[0], f"{paths[1]}/{filename[1]}")

    for filename in testFilenames:
        copyfile(filename[0], f"{paths[2]}/{filename[1]}")

# Setup thư mục chứa data

In [ ]:
def formatFilenameFSDD(dir):
    filenames = []
    count = 0
    for filename in os.listdir(dir):
        tmp = str(count) + '_' + filename.split('_')[0] + '.wav'
        filenames.append((f"{dir}/{filename}", tmp))
        count += 1
    return filenames

filenames = formatFilenameFSDD('./dataset_2_FSDD')
makeTrainTestDir('fsdd', filenames, fsdd_split)

In [ ]:
def formatFilenameWolfram(dir):
    filenames = []
    count = 0
    folders = os.listdir(dir)
    for folder in folders:
        for filename in os.listdir(f'{dir}/{folder}'):
            tmp = str(count) + '_' + folder.split('_')[0] + '.wav'
            filenames.append((f"{dir}/{folder}/{filename}", tmp))
            count += 1

    return filenames

filenames = formatFilenameWolfram('./dataset_1_wolfram')
makeTrainTestDir('wolfram', filenames, wolfram_split)

# Huấn luyện mô hình (có thể bỏ qua vì mô hình đã save trên drive)

## Hàm train mô hình

In [ ]:
def Generate_DiagGMM(full_fset, n_features, n_states=5, n_cmps=3):
  means = np.mean(full_fset, axis=0)
  stds = np.std(full_fset, axis=0)
  # initial values for all gaussian components
  np.random.seed(None)
  dist_init = np.random.random((n_states, n_cmps, n_features, 2))
  dist_init[..., 0] -= 0.5  # center means to 0.0
  #for feat_i in range(n_features):
      # random init mean in range [-std, std)
      #dist_init[..., feat_i, 0] *= 2 * stds[feat_i]
      #dist_init[..., feat_i, 0] += means[feat_i]
      # random init std in range 1std/n_components
      #dist_init[..., feat_i, 1] *= stds[feat_i] / n_cmps

  dists = tuple(
      GeneralMixtureModel(list(
          IndependentComponentsDistribution(tuple(
              NormalDistribution(*dist_init[state_i, cmp_i, feat_i, :])
              for feat_i in range(n_features)
          ))
          for cmp_i in range(n_cmps)
      ))
      if n_cmps > 1 else
      IndependentComponentsDistribution(tuple(
          NormalDistribution(*dist_init[state_i, 0, feat_i, :])
          for feat_i in range(n_features)
      ))
      for state_i in range(n_states)
  )
  return dists

In [ ]:
# Generate progressive HMM model
def left_right_GMMHMM(seed_sample, x_dim, n_states=10, n_modals=9, random=0):
  rng = np.random.RandomState(random)
  if random!=0:
    init_prob = lambda: rng.rand(1)[0]
    
  model = HiddenMarkovModel()
  states = [ ]
  for i, state_model in enumerate(Generate_DiagGMM(seed_sample, x_dim, n_states, n_modals)):
    state = State( state_model, name=f"H{i}" )
    model.add_state(state)
    states.append(state)
  model.add_transition(model.start, states[0], 1)
  for i in range(n_states-1):
    model.add_transition(states[i], states[i], 0.5)
    model.add_transition(states[i], states[i+1], 0.5)
  model.add_transition(states[n_states-1], states[n_states-1], 0.5)
  model.add_transition(states[n_states-1], model.end, 0.5)
  model.bake()
  return model

In [ ]:
def train_GMMHMM(dataset, input_dim, n_hidden_state, n_gauss_modal):
    GMMHMM_Models = dict()

    for label in dataset.keys():
        print(f"Training model detect {label}")
        model = left_right_GMMHMM(dataset[label][0], input_dim, n_hidden_state, n_gauss_modal)
        model.fit(dataset[label], verbose=True, multiple_check_input=False)  # get optimal parameters
        GMMHMM_Models[label] = model

    return GMMHMM_Models

## Hàm test mô hình

In [ ]:
# test model
def evaluateModel(testDataset, model):
  if (len(testDataset) == 0):
    return
  digit_clf = BayesClassifier(list(dict(sorted(model.items())).values()))
  true_cnt = 0
  total = 0
  for label in sorted(testDataset.keys()):
      features = np.array(testDataset[label], dtype='object')
      pred = digit_clf.predict(features)
      iter_cnt = (pred == int(label)).sum()
      iter_total = len(features)
      total += iter_total
      true_cnt += iter_cnt
      print(f"{label}: {iter_cnt}/{iter_total} ({iter_cnt/iter_total})")
  print("Final recognition rate is %.2f"%(100.0*true_cnt/total), "%")

## Đọc và tiền xử lý data

In [ ]:
# prepare data for training
master_path = 'fsdd'

trainDir = master_path + '/train_audio/'
trainDataSet_fsdd = buildDataSet(trainDir)
print("Finish prepare the training data")

# prepare data for testing
testDir = master_path + '/test_audio/'
testDataSet_fsdd = buildDataSet(testDir)
print("Finish prepare the test data")

Finish prepare the training data
Finish prepare the test data


In [ ]:
# prepare data for training
master_path = 'wolfram'

trainDir = master_path + '/train_audio/'
trainDataSet_wolfram = buildDataSet(trainDir)
print("Finish prepare the training data")

# prepare data for testing
testDir = master_path + '/test_audio/'
testDataSet_wolfram = buildDataSet(testDir)
print("Finish prepare the test data")

Finish prepare the training data
Finish prepare the test data


In [ ]:
trainDataSet = trainDataSet_fsdd
if wolfram_split > 0:
  for label in trainDataSet_fsdd.keys():
    trainDataSet[label] = trainDataSet_fsdd[label] + trainDataSet_wolfram[label]

## **Mô hình 1**
- 10 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
for label in trainDataSet.keys():
  print(min(map(len, trainDataSet[label])))

10
10
10
10
10
10
10
10
10
10


In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,10,n_mixtures)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 5461736.1419194145	Time (s): 0.04818
[2] Improvement: 1475.6001369551523	Time (s): 0.04713
[3] Improvement: 605.9464692353795	Time (s): 0.04994
[4] Improvement: 279.53297423233744	Time (s): 0.04935
[5] Improvement: 121.78784178235219	Time (s): 0.04326
[6] Improvement: 46.82062427143683	Time (s): 0.04258
[7] Improvement: 11.39270647338708	Time (s): 0.04646
[8] Improvement: 6.243042443325976	Time (s): 0.0466
[9] Improvement: 3.4087930849636905	Time (s): 0.04276
[10] Improvement: 2.1015792543184943	Time (s): 0.04624
[11] Improvement: 1.2697855466685724	Time (s): 0.04217
[12] Improvement: 0.4430183911463246	Time (s): 0.04567
[13] Improvement: 0.04573984301532619	Time (s): 0.04705
[14] Improvement: 0.0036930434871464968	Time (s): 0.05285
[15] Improvement: 0.00042018218664452434	Time (s): 0.05061
[16] Improvement: 5.9719139244407415e-05	Time (s): 0.04631
[17] Improvement: 9.797950042411685e-06	Time (s): 0.04676
[18] Improvement: 1.7457350622862577e-06

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (105/117)
2: (198/233)
7: (284/327)
6: (397/470)
8: (488/599)
0: (609/721)
9: (714/840)
5: (839/970)
3: (912/1074)
4: (1020/1200)
Final recognition rate is 85.00 %
Evaluate on rest of FSDD set
0: (173/178)
3: (296/374)
9: (453/555)
7: (636/761)
4: (797/935)
5: (963/1105)
8: (1075/1276)
1: (1246/1459)
2: (1406/1643)
6: (1524/1800)
Final recognition rate is 84.67 %
Evaluate on wolfram test set
4: (1319/2372)
8: (2471/4724)
7: (3249/7101)
1: (4748/9471)
2: (5974/11844)
3: (7209/14200)
6: (7971/16569)
0: (9251/18945)
5: (10750/21302)
9: (12712/23666)
Final recognition rate is 53.71 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_10_{n_mixtures}[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()

## **Mô hình 2**
- 5 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,5,n_mixtures)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 1057924.1986410776	Time (s): 0.0238
[2] Improvement: 1824.7478207260283	Time (s): 0.02367
[3] Improvement: 649.0267748781771	Time (s): 0.02409
[4] Improvement: 125.371035593329	Time (s): 0.02385
[5] Improvement: 64.50156896290719	Time (s): 0.02382
[6] Improvement: 53.65640420463751	Time (s): 0.02522
[7] Improvement: 39.8688242638309	Time (s): 0.02444
[8] Improvement: 48.401963941782014	Time (s): 0.02829
[9] Improvement: 58.87595085761859	Time (s): 0.0261
[10] Improvement: 90.23974324812298	Time (s): 0.02576
[11] Improvement: 73.32193011030904	Time (s): 0.0249
[12] Improvement: 59.534856144251535	Time (s): 0.02374
[13] Improvement: 32.8288965014508	Time (s): 0.02361
[14] Improvement: 22.630808104149764	Time (s): 0.02464
[15] Improvement: 13.748554079793394	Time (s): 0.02491
[16] Improvement: 12.28316631988855	Time (s): 0.02746
[17] Improvement: 16.318472885817755	Time (s): 0.02685
[18] Improvement: 22.73707858589478	Time (s): 0.02617
[19] Improve

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (95/117)
2: (137/233)
7: (215/327)
6: (333/470)
8: (441/599)
0: (559/721)
9: (662/840)
5: (790/970)
3: (863/1074)
4: (975/1200)
Final recognition rate is 81.25 %
Evaluate on rest of FSDD set
0: (162/178)
3: (275/374)
9: (427/555)
7: (598/761)
4: (760/935)
5: (927/1105)
8: (1068/1276)
1: (1218/1459)
2: (1287/1643)
6: (1407/1800)
Final recognition rate is 78.17 %
Evaluate on wolfram test set
4: (1215/2372)
8: (2614/4724)
7: (3421/7101)
1: (4752/9471)
2: (5399/11844)
3: (6602/14200)
6: (7226/16569)
0: (7924/18945)
5: (9354/21302)
9: (11136/23666)
Final recognition rate is 47.05 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_5_{n_mixtures}[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()

## **Mô hình 3**
- 3 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,3,n_mixtures)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 320679.7557637595	Time (s): 0.01884
[2] Improvement: 595.6459858104063	Time (s): 0.01652
[3] Improvement: 114.15391274106514	Time (s): 0.01661
[4] Improvement: 138.97949758409231	Time (s): 0.01652
[5] Improvement: 361.5293756878673	Time (s): 0.01704
[6] Improvement: 352.31156910100253	Time (s): 0.01753
[7] Improvement: 434.1709712656884	Time (s): 0.01717
[8] Improvement: 455.2816719049879	Time (s): 0.01653
[9] Improvement: 98.89663764876605	Time (s): 0.01672
[10] Improvement: 13.036074307266972	Time (s): 0.01648
[11] Improvement: 4.645940477232216	Time (s): 0.02053
[12] Improvement: 7.789586769795278	Time (s): 0.01688
[13] Improvement: 10.905006823682925	Time (s): 0.01661
[14] Improvement: 4.636793412515544	Time (s): 0.01654
[15] Improvement: 2.049897662116564	Time (s): 0.01625
[16] Improvement: 1.3357234026334481	Time (s): 0.01672
[17] Improvement: 1.2049490953359054	Time (s): 0.01663
[18] Improvement: 0.9138794703758322	Time (s): 0.02151
[19] 

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (99/117)
2: (198/233)
7: (274/327)
6: (284/470)
8: (412/599)
0: (531/721)
9: (631/840)
5: (710/970)
3: (785/1074)
4: (901/1200)
Final recognition rate is 75.08 %
Evaluate on rest of FSDD set
0: (173/178)
3: (295/374)
9: (445/555)
7: (605/761)
4: (773/935)
5: (899/1105)
8: (1051/1276)
1: (1208/1459)
2: (1371/1643)
6: (1383/1800)
Final recognition rate is 76.83 %
Evaluate on wolfram test set
4: (1087/2372)
8: (2251/4724)
7: (2882/7101)
1: (4099/9471)
2: (5093/11844)
3: (6106/14200)
6: (6381/16569)
0: (7442/18945)
5: (8569/21302)
9: (10454/23666)
Final recognition rate is 44.17 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_3_{n_mixtures}[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()